## Метод главных компонент: Составление фондового индекса

In [1]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA

In [2]:
def write_answer(filename, answer):
    
    with open(filename, 'w') as fout:
        
        fout.write(str(answer))
        fout.close()

### Введение

<b>Метод главных компонент (principal component analysis, PCA)</b> — это один из методов обучения без учителя, который позволяет сформировать новые признаки, являющиеся линейными комбинациями старых. При этом новые признаки строятся так, чтобы сохранить как можно больше дисперсии в данных. Иными словами, метод главных компонент понижает размерность данных оптимальным с точки зрения сохранения дисперсии способом.

Основным параметром метода главных компонент является количество новых признаков. Как и в большинстве методов машинного обучения, нет четких рекомендаций по поводу выбора значения этого параметров. Один из подходов — выбирать минимальное число компонент, при котором объясняется не менее определенной доли дисперсии (это означает, что в выборке сохраняется данная доля от исходной дисперсии).

В этом задании понадобится измерять схожесть двух наборов величин. Если имеется набор пар измерений (например, одна пара — предсказания двух классификаторов для одного и того же объекта), то охарактеризовать их зависимость друг от друга можно с помощью корреляции Пирсона. Она принимает значения от -1 до 1 и показывает, насколько данные величины линейно зависимы. Если корреляция равна -1 или 1, то величины линейно выражаются друг через друга. Если она равна нулю, то линейная зависимость между величинами отсутствует.

### Данные

В этом задании мы будем работать с данными о стоимостях акций 30 крупнейших компаний США. На основе этих данных можно оценить состояние экономики, например, с помощью индекса Доу-Джонса. Со временем состав компаний, по которым строится индекс, меняется. Для набора данных был взят период с 23.09.2013 по 18.03.2015, в котором набор компаний был фиксирован (подробнее почитать о составе можно по ссылке из материалов). 

Одним из существенных недостатков индекса Доу-Джонса является способ его вычисления — при подсчёте индекса цены входящих в него акций складываются, а потом делятся на поправочный коэффициент. В результате, даже если одна компания заметно меньше по капитализации, чем другая, но стоимость одной её акции выше, то она сильнее влияет на индекс. Даже большое процентное изменение цены относительно дешёвой акции может быть нивелировано незначительным в процентном отношении изменением цены более дорогой акции.

### Реализация в sklearn

Метод главных компонент реализован в пакете scikit-learn в модуле decomposition в классе [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). Основным параметром является количество компонент (n_components). Для обученного преобразования этот класс позволяет вычислять различные характеристики. Например, поле explained_variance_ratio_ содержит процент дисперсии, который объясняет каждая компонента. Поле components_ содержит информацию о том, какой вклад вносят признаки в компоненты. Чтобы применить обученное преобразование к данным, можно воспользоваться методом transform. 

Для нахождения коэффициента корреляции Пирсона можно воспользоваться функцией [corrcoef](https://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.corrcoef.html) из пакета numpy.

### Материалы

- [Dow Jones Industrial Average](https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average)
- [История состава компаний, входящих в индекс Dow Jones](https://en.wikipedia.org/wiki/Historical_components_of_the_Dow_Jones_Industrial_Average)

Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.42. При необходимости округляйте дробную часть до двух знаков.

#### 1. Загрузите данные close_prices.csv. В этом файле приведены цены акций 30 компаний на закрытии торгов за каждый день периода. 

In [3]:
X = pd.read_csv('close_prices.csv', index_col='date')
X.head()

,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,HD,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
date,,,,,,,,,,,,,,,,,,,,,
2013-09-23,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,75.910004,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
2013-09-24,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,76.040001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2013-09-25,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,75.519997,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
2013-09-26,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,76.070000,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
2013-09-27,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,75.959999,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002


#### 2. На загруженных данных обучите преобразование PCA с числом компоненты равным 10. Скольких компонент хватит, чтобы объяснить 90% дисперсии?

In [4]:
# fit PCA transformation with 10 components

pca = PCA(n_components=10)
X_pca = pca.fit(X)

In [5]:
var = X_pca.explained_variance_ratio_
var

array([0.73897118, 0.11007169, 0.04995088, 0.0287492 , 0.02215448,
       0.01931577, 0.00674853, 0.00614091, 0.00320594, 0.00305611])

In [6]:
num_components = np.argmax(var.cumsum() > 0.9) + 1
num_components

4

In [7]:
write_answer('submission_pca_1.txt', num_components)

Посчитав кумулятивные суммы, можно видеть, что 3 компоненты объясняют 89.9% дисперсии, а 4 - 92.77%

#### 3. Примените построенное преобразование к исходным данным и возьмите значения первой компоненты. 

In [8]:
X_transformed = pca.transform(X)
first_comp = X_transformed[:, 0]

#### 4. Загрузите информацию об индексе Доу-Джонса из файла djia_index.csv. Чему равна корреляция Пирсона между первой компонентой и индексом Доу-Джонса? 

In [9]:
dow_jones = pd.read_csv('djia_index.csv', index_col='date')
dow_jones.head()

,^DJI
date,
2013-09-23,15401.379883
2013-09-24,15334.589844
2013-09-25,15273.259766
2013-09-26,15328.299805
2013-09-27,15258.240234


In [10]:
corr = np.corrcoef(first_comp, dow_jones['^DJI'])[0, 1]
corr

0.9096522193050237

In [11]:
write_answer('submission_pca_2.txt', round(corr, 2))

Можно увидеть, что коэффициент корреляции получился достаточно большим.

#### 5. Какая компания имеет наибольший вес в первой компоненте? Укажите ее название с большой буквы.

In [12]:
idx = np.argmax(X_pca.components_[0])
idx

26

In [13]:
name = X.columns.values[idx]
name

'V'

Это сокращенное название компании Visa.

In [14]:
write_answer('submission_pca_3.txt', name)